<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2021-1/Insumos/SWAPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"



/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 594 (delta 7), reused 0 (delta 0), pack-reused 579
Receiving objects: 100% (594/594), 32.97 MiB | 31.67 MiB/s, done.
Resolving deltas: 100% (386/386), done.


In [2]:
%%R
#require(quantmod)
#require(data.table)
install.packages("data.table")
library(data.table)
#require("PerformanceAnalytics")



library(dplyr)


R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/data.table_1.13.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 5273400 bytes (5.0 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R

In [3]:
%%R
#FUNCIÓN DE INTERPOLACIÓN ALAMBRADA

talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}



# Definición de parámetros para valorar:
SWAPS


Cargamos parámetros para valoración:

In [12]:
%%R
fval=as.Date("20180628",format="%Y%m%d") #Fecha de valoración
itpl=0 #0 lineal 1 interpolación alambrada

#SWAP
btasadesc_sw="RiesgosFinancieros/2021-1/Insumos/tasa_TIIE_SW_OP.txt"
btasacupvar_sw="RiesgosFinancieros/2021-1/Insumos/tasa_DIRS_SW_OP.txt"
tasafija_sw=cbind(0.079,0.075) #se establece la tasa fija a pagar para cada swap
plazos_sw=cbind(588,360) #se establece el número de días que vivirá el swap
plazocupon_sw=cbind(28,28) #se establece el número de días que se pagará cada cupón
contratos_sw=cbind(-1600,100000) #se establece el número de contratos_sw de cada swap
nominal_sw=cbind(1,1) #se establece el nominal_sw de cada swap
por_sw=cbind(0,0) #se establece 0 si se paga tasa fija y 1 si se paga tasa variable



# Carga de datos

In [13]:
%%R

  
  #CARGA DE DATOS PARA SWAP
  data1<-read.table(btasadesc_sw)
  n1<-nrow(data1)
  m1_orig_sw=ncol(data1)
  X1_orig_sw=data.table(mutate(data1[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
  X1_orig_sw=X1_orig_sw%>%select(-V1)
  nodos1_sw=data.frame(data1[1,2:m1_orig_sw])
  
  data2<-read.table(btasacupvar_sw)
  n2<-nrow(data2)
  m2_orig_sw=ncol(data2)
  X2_orig_sw=data.table(mutate(data2[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
  X2_orig_sw=X2_orig_sw%>%select(-V1)
  nodos2_sw=data.frame(data2[1,2:m2_orig_sw])
  

print(X1_orig_sw[1:4])
print(X2_orig_sw[1:4])
print(nodos1_sw)

         V2       V3       V4       V5       V6       V7       V8       V9
1: 8.051485 8.062321 8.104358 8.223960 8.393201 8.492391 8.572089 8.770682
2: 8.016536 8.048647 8.102803 8.224503 8.401055 8.492246 8.572144 8.751695
3: 8.050352 8.074009 8.102297 8.232799 8.400874 8.492216 8.572122 8.751831
4: 8.001825 8.038139 8.105294 8.223375 8.384518 8.491140 8.563166 8.752018
        V10      V11      V12       V13      V14      V15      V16      V17
1: 8.930412 9.232427 9.610588 10.082187 10.59112 11.14255 11.73502 12.37160
2: 8.911093 9.181149 9.522809  9.976765 10.46750 11.01064 11.59501 12.22267
3: 8.891261 9.181133 9.523522  9.977425 10.46810 11.00649 11.58510 12.20624
4: 8.881459 9.191721 9.524074  9.995334 10.50682 11.03940 11.60882 12.21951
         Date
1: 2020-03-06
2: 2020-03-05
3: 2020-03-04
4: 2020-03-03
         V2       V3       V4       V5       V6       V7       V8       V9
1: 7.900000 8.000000 8.102531 8.166071 8.247143 8.258393 8.250357 8.113077
2: 7.917478 8.017699 8.12

# Integración de Insumos

Aseguramiento de que todos los factores de riesgo estén alineados en sus fechas, nos quedamos con la intersección. Posteriormente, filtramos cada histórico factor de riesgo hacia esta intersección de todos.

In [14]:
%%R
#INTERSECCIÓN DE FECHAS DE TODOS LOS INSUMOS    
  
#head(x_orig_gov)  
sw_dt=data.table(Date=as.Date(X1_orig_sw[X2_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón
print(sw_dt)

           Date
  1: 2020-03-06
  2: 2020-03-05
  3: 2020-03-04
  4: 2020-03-03
  5: 2020-03-02
 ---           
228: 2019-03-25
229: 2019-03-21
230: 2019-03-20
231: 2019-03-19
232: 2019-03-18


In [15]:
%%R
n=nrow(sw_dt) #Historia de todos



#historia de swaps
X1_orig_sw=sw_dt[X1_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X1_orig_sw=X1_orig_sw%>%select(-Date)/100
X2_orig_sw=sw_dt[X2_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X2_orig_sw=X2_orig_sw%>%select(-Date)/100


print(X1_orig_sw[1:3])

           V2         V3         V4         V5         V6         V7         V8
1: 0.08051485 0.08062321 0.08104358 0.08223960 0.08393201 0.08492391 0.08572089
2: 0.08016536 0.08048647 0.08102803 0.08224503 0.08401055 0.08492246 0.08572144
3: 0.08050352 0.08074009 0.08102297 0.08232799 0.08400874 0.08492216 0.08572122
           V9        V10        V11        V12        V13       V14       V15
1: 0.08770682 0.08930412 0.09232427 0.09610588 0.10082187 0.1059112 0.1114255
2: 0.08751695 0.08911093 0.09181149 0.09522809 0.09976765 0.1046750 0.1101064
3: 0.08751831 0.08891261 0.09181133 0.09523522 0.09977425 0.1046810 0.1100649
         V16       V17
1: 0.1173502 0.1237160
2: 0.1159501 0.1222267
3: 0.1158510 0.1220624


## SWAPS

In [18]:
%%R
##SWAP TASA FIJA VS TASA VARIABLE CÁLCULO

##Interpolamos

nodosvp=nodos1_sw
nodostc=nodos2_sw
curvavp=as.matrix(X1_orig_sw)
curvatc=X2_orig_sw
n1=nrow(curvavp)
n2=nrow(curvatc)


m=max(ncol(plazos_sw),1) #número de contratos_sw swap a valorar
print(m)
N=matrix(0,1,m) #es un vector de m valores donde se cargarán los m número de cupones a pagar para cada contrato

for (j in (1:m))
{
  N[j]=as.integer(plazos_sw[j]/plazocupon_sw[j])+1 #número de cupones a pagar
}
print(N)
VTplazos_sw=matrix(0,1,sum(N)) #vector de todos los plazos_sw de todos los contratos_sw
contratos_swT=matrix(0,1,sum(N)) #vector de todos los contratos_sw de todos los flujos de todos los contratos_sw
nominal_swT=matrix(0,1,sum(N)) #vector de todos los nominal_swes de todos los flujos de todos los contratos_sw
por_swT=matrix(0,1,sum(N)) #vector de todos los dummy si paga o recibe de todos los flujos de todos los contratos_sw
plazocupon_swT=matrix(0,1,sum(N)) #vector de todos los plazos_swcupon de todos los flujos de todos los contratos_sw
tasafija_swT=matrix(0,1,sum(N)) #vector de tasas fijas de todos los flujos de todos los contratos_sw
VTplazos_swc=matrix(0,1,sum(N)) #vector de todos los plazos_sw cortos de todos los contratos_sw


plazini=plazos_sw-plazocupon_sw*(N-1) #vector de plazos_sw iniciales}
print(plazini)


[1] 2
     [,1] [,2]
[1,]   22   13
     [,1] [,2]
[1,]    0   24


In [20]:
%%R
for (j in (1:m))
{
  if (j==1)
  {
    VTplazos_sw[,1:sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,1:sum(N[1:j])]=c(0,VTplazos_sw[,1:(sum(N[1:j])-1)])
    contratos_swT[,1:sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])
    nominal_swT[,1:sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,1:sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,1:sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,1:sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
  else
  {
    VTplazos_sw[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,(sum(N[1:j-1])+1):sum(N[1:j])]=c(0,VTplazos_sw[,(sum(N[1:j-1])+1):(sum(N[1:j])-1)])
    contratos_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])					
    nominal_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
}

print(VTplazos_sw)
print(VTplazos_swc)

     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0   28   56   84  112  140  168  196  224   252   280   308   336   364
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]   392   420   448   476   504   532   560   588    24    52    80   108
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35]
[1,]   136   164   192   220   248   276   304   332   360
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    0   28   56   84  112  140  168  196   224   252   280   308   336
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]   364   392   420   448   476   504   532   560     0    24    52    80
     [,27] [,28] [,29] [,30] [,31] [,32] [,33] [,34] [,35]
[1,]   108   136   164   192   220   248   276   304   332


In [24]:
%%R
Xvp=matrix(0,n,ncol(VTplazos_sw))
Xtc=matrix(0,n,ncol(VTplazos_sw))
Xtcc=matrix(0,n,ncol(VTplazos_sw))
XtfwdT=matrix(0,n,ncol(VTplazos_sw))

for (i in (1:n))
{
  Xvp[i,]=if(itpl==0){approx(nodosvp,curvavp[i,],VTplazos_sw,rule=2)$y}else{talamb(nodosvp,curvavp[i,],VTplazos_sw)}
  Xtc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazos_sw,rule=2)$y}else{talamb(nodostc,curvatc[i,],VTplazos_sw)}
  Xtcc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazos_swc, rule=2)$y}else{talamb(nodostc,curvatc[i,],VTplazos_swc)}
  
  XtfwdT[i,]=((1+Xtc[i,]*VTplazos_sw/360)/(1+Xtcc[i,]*VTplazos_swc/360)-1)*360/plazocupon_swT 
  for (j in (1:ncol(VTplazos_sw)))
  {
    if (VTplazos_sw[j]<= plazocupon_swT[j])
    {
      XtfwdT[i,j]=Xtc[i,j]
    }
    else
    {
      j=sum(N[1:j])
    }
  }
}

print(Xvp[1:28])
print(XtfwdT[1:10])

 [1] 0.08051485 0.08016536 0.08050352 0.08001825 0.08031415 0.08013452
 [7] 0.08014507 0.08031415 0.08031415 0.07997598 0.08043033 0.08039869
[13] 0.08065231 0.08048323 0.08039869 0.08013452 0.08018734 0.08012309
[19] 0.08010280 0.08004362 0.07996548 0.07992526 0.08006053 0.08004362
[25] 0.07997598 0.08017677 0.08014507 0.08001825
 [1] 0.07900000 0.07917478 0.07934956 0.07865044 0.07865044 0.07865044
 [7] 0.07865044 0.07882522 0.07847566 0.07865044


In [25]:
%%R

X_sw=cbind(XtfwdT,Xvp)

swap=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp, N)
{
  V0=matrix(0,1,ncol(N))
  V0f=(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT/360)))/(1+Xvp*VTplazos_sw/360))*nominal_swT*(-1)^por_swT
  for (j in (1:ncol(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
}

V0_sw=swap(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,], N)

V0_sw

         [,1]     [,2]
[1,] 4.875753 442.4803
